<h1>Recommendations</h1>

<ul>
  <li><a href="#Anime">Anime</a></li>
  <li><a href="#Manga">Manga</a></li>
</ul>

In [ ]:
username = "Fro116"
source = "MAL"
DEBUG = false;

In [ ]:
import NBInclude: @nbinclude
@nbinclude("RecommendationsBase.ipynb");

# Anime

In [ ]:
recommend("anime")

# Manga

In [ ]:
recommend("manga")